In [3]:
import requests
import os
import time
import bs4
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import json

service = Service()
driver = webdriver.Chrome(service=service)
# 讀取txt文件中的URL
with open('心理測驗.txt', 'r', encoding='utf-8') as file:
#with open('心理學概論.txt', 'r', encoding='utf-8') as file:
#with open('心理學理論.txt', 'r', encoding='utf-8') as file:
#with open('大眾心理學.txt', 'r', encoding='utf-8') as file:
#with open('個案分析與諮商輔導.txt', 'r', encoding='utf-8') as file:
#with open('心理學工具書.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 初始化一個空字典來儲存書名和URL
urls = {}
current_title = ''

for line in lines:
    line = line.strip()
    if line.startswith('http'):
        urls[current_title] = line
    else:
        current_title = line
        
all_books_data=pd.DataFrame()
# 遍歷每個URL，爬取內容並寫入單獨的文件
for title, url in urls.items():
    res = requests.get(url)
    driver.get(url) 
    WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.publicDate.d-flex.mb-1")))
    product_items = driver.find_elements(By.CSS_SELECTOR, "div.group")
    
    info = {}
    current_key = None

    for item in product_items:
        spans = item.find_elements(By.TAG_NAME, "span")
        text = ''.join([span.text for span in spans]).strip()

        if text.endswith('：'):
            current_key = text[:-1]  # 移除結尾的冒號
        elif current_key:
            info[current_key] = text
            current_key = None
    #info{作者,譯者,出版社,出版日期}
       
    book_data = {
        '書名':title,
        'URL': url,
        '作者介紹': '',
        '各界推薦': '',
        '推薦序': '',
        # 根據需要添加更多的欄位
    }   
    
    soup = BeautifulSoup(res.text, 'html.parser')
    content = soup.find_all("p")
    for p in content:
        text = p.get_text()
        #print(p.get_text() + '\n' + '\n')
        if '作者介紹' in text:
            book_data['作者介紹'] = text.replace('作者介紹', '').strip()
        elif '產品目錄' in text:
            book_data['產品目錄'] = text.replace('產品目錄', '').strip()
        elif '內容簡介' in text:
            book_data['內容簡介'] = text.replace('內容簡介', '').strip()
        elif '各界推薦' in text:
            book_data['各界推薦'] = text.replace('各界推薦', '').strip()
        elif '自序' in text:
            book_data['自序'] = text.replace('自序 :', '').strip()
        elif '內文' in text:
            book_data['內文'] = text.replace('內文 :', '').strip()
        elif '推薦序' in text:
            book_data['推薦序'] = text.replace('推薦序', '').strip()
        elif '最佳賣點' in text:
            book_data['最佳賣點'] = text.replace('最佳賣點 :', '').strip()
    
    #合併info和book_data兩個dictionary
    book_data.update(info)
    filename = f"{title.replace('?', '').replace('*', '').replace(':', '').replace(' ', '').replace('/', '-')}.json"
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(book_data, file, ensure_ascii=False, indent=4)
print("完成")

完成


In [5]:
df1 = pd.read_excel('book_data_心理測驗.xlsx')
df2 = pd.read_excel('book_data_心理學工具書.xlsx')

In [6]:
df3 = pd.concat([df1,df2], ignore_index=True)

In [7]:
df3.to_excel('測驗+工具書.xlsx',index=True)